In [3]:
#importando as bibliotecas que iremos usar
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

#Lista de capitais para entrega no Nordeste
capitais = ["Fortaleza", "Natal", "João Pessoa", "Maceió", "Aracaju", "Teresina", "São Luís"]
#População de outras capitais no Nordeste, segundo censo de 2022 do IBGE, ordenadas tal qual a lista capitais
capitais_pop = [2428708, 751300, 833932, 957916, 602757, 866300, 1037775]
#PIB per capita de outras capitais do Nordeste, segundo censo de 2021 do IBGE, ordenadas tal qual a lista capitais
capitais_pib = [27164.45, 26972.28, 26936.78, 26642.20, 27364.40, 27430.28, 32739.65]

#Tempo (em minutos) entre Recife/Salvador e outras capitais do Nordeste, segundo site Rotas Brasil, ordenados tal qual a lista capitais
recife_distancias = [687, 238, 104, 192, 378, 1011, 1424]
salvador_distancias = [918, 853, 719, 466, 268, 1018, 1432]

#Tempo médio (em minutos) entre Recife/Salvador e outras capitais, ponderado pela população destas capitais
recife_distancia_ponderada = 0
salvador_distancia_ponderada = 0
pesos = 0
for i in range(0, len(capitais)):
  recife_distancia_ponderada += recife_distancias[i] * capitais_pop[i]
  salvador_distancia_ponderada += salvador_distancias[i] * capitais_pop[i]
  pesos = pesos + capitais_pop[i]
recife_distancia_ponderada = recife_distancia_ponderada/pesos
salvador_distancia_ponderada = salvador_distancia_ponderada/pesos

#Tempo médio (em minutos) entre Recife/Salvador e outras capitais, ponderado pelo PIB per capita destas capitais
recife_distancia_pib = 0
salvador_distancia_pib = 0
pesos = 0
for i in range(0, len(capitais)):
  recife_distancia_pib += recife_distancias[i] * capitais_pib[i]
  salvador_distancia_pib += salvador_distancias[i] * capitais_pib[i]
  pesos = pesos + capitais_pib[i]
recife_distancia_pib = recife_distancia_pib/pesos
salvador_distancia_pib = salvador_distancia_pib/pesos

#Custo médio por m2 (R$/m2)
recife_custo = 7647
salvador_custo = 8293

#População em Recife/Salvador. Representam potenciais consumidores diretos, ou potencial mão de obra
recife_pop = 1488920
salvador_pop = 2417678

#Criar DataFrame do pandas usando os dados obtidos anteriormente
data = {
    "cidade": ["Recife", "Salvador"],
    "distancia_ponderada": [recife_distancia_ponderada, salvador_distancia_ponderada],
    "distancia_pib": [recife_distancia_pib, salvador_distancia_pib],
    "custo_m2": [recife_custo, salvador_custo],
    "populacao":[recife_pop, salvador_pop]
}

df = pd.DataFrame(data)

#Inverter variáveis onde maior é melhor (transformar para variáveis onde menor é melhor, necessário para o modelo)
df["distancia_pib_invertida"] = df["distancia_pib"].max() - df["distancia_pib"]

#Selecionar colunas relevantes para a análise
features = ["distancia_ponderada", "distancia_pib_invertida", "custo_m2", "populacao"]

#Normalizar para escala 0-1
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(df[features])

#Aplicar pesos
pesos = np.array([0.25, 0.25, 0.25, 0.25])
score = X_scaled.dot(pesos)

#Adicionar score ao DataFrame
df["score_final"] = score

#Mostrar resultado final
df_resultado = df[["cidade", "score_final"]].sort_values(by="score_final")
print(df_resultado)


[[0. 1. 0. 0.]
 [1. 0. 1. 1.]]
     cidade  score_final
0    Recife         0.25
1  Salvador         0.75
